<a href="https://colab.research.google.com/github/hellwalker3/naive-txt2img-api-of-sd/blob/main/Extracting_Pose_Estimation_from_Videos_using_MMpose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Mount Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Install Required Dependencies
!pip install yt-dlp 
# install dependencies: (use cu111 because colab has CUDA 11.1)
%pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install MMEngine, MMCV and MMDetection using MIM
%pip install -U openmim
!mim install mmengine
!mim install "mmcv>=2.0.0rc1"
!mim install "mmdet>=3.0.0rc0"

!git clone https://github.com/open-mmlab/mmpose.git -b 1.x
# "-b 1.x" means checkout to the `1.x` branch.
%cd mmpose
# for better Colab compatibility, install xtcocotools from source%pip install git+https://github.com/jin-s13/xtcocoapi%pip install -r requirements.txt
%pip install -v -e .
# "-v" means verbose, or more output
# "-e" means installing a project in editable mode,
%cd ..

import sys
sys.path.append('/content/mmpose')

!git clone https://github.com/TencentARC/T2I-Adapter.git
!cd T2I-Adapter && !git checkout 7056c7f

!pip install safetensors
!pip uninstall numpy -y
!pip install numpy==1.23.5

In [ ]:
#@title Note: You need to restart the runtime here

In [ ]:
#@title Set Output Folder Name and URL

output_folder_name = ""
youtube_url = ""
fps = 30

In [ ]:
#@title Extract Images from Video

import os
import shutil

def create_folder(folder_path):
    """
    Remove the existing folder if it exists, and create the folder if it does not exist.
    """
    # Remove the existing folder if it exists, even if it contains files
    if os.path.exists(folder_path):
        print("Removing existing folder...")
        shutil.rmtree(folder_path)

    # Create the folder
    print("Creating folder...")
    os.mkdir(folder_path)


create_folder('input_images')


command = f"yt-dlp --get-filename -o 'movie.%(ext)s' '{youtube_url}'"
file_name = os.popen(command).read().strip()

!yt-dlp -o "movie.%(ext)s" {youtube_url}
!ffmpeg -i {file_name} -vf fps={fps} input_images/img%d.png
!rm {file_name}

In [ ]:
#@title Save Skeleton Images using mmpose
%cd /content/T2I-Adapter

def create_folder(folder_path):
    """
    Remove the existing folder if it exists, and create the folder if it does not exist.
    """
    # Remove the existing folder if it exists, even if it contains files
    if os.path.exists(folder_path):
        print("Removing existing folder...")
        shutil.rmtree(folder_path)

    # Create the folder
    print("Creating folder...")
    os.mkdir(folder_path)

create_folder(f'/content/drive/MyDrive/{output_folder_name}')

import cv2
import matplotlib.pyplot as plt
import glob, os
import numpy as np
from ldm.modules.extra_condition.utils import skeleton, pose_kpt_color, pose_link_color 


def imshow_keypoints(img,
                     pose_result,
                     kpt_score_thr=0.1,
                     radius=2,
                     thickness=2):
    """Draw keypoints and links on an image.
    Args:
            img (ndarry): The image to draw poses on.
            pose_result (list[kpts]): The poses to draw. Each element kpts is
                a set of K keypoints as an Kx3 numpy.ndarray, where each
                keypoint is represented as x, y, score.
            kpt_score_thr (float, optional): Minimum score of keypoints
                to be shown. Default: 0.3.
            thickness (int): Thickness of lines.
    """

    img_h, img_w, _ = img.shape
    img = np.zeros(img.shape)

    for idx, kpts in enumerate(pose_result):
        if idx > 0:
            continue
        kpts = kpts.pred_instances['keypoints'][0]
        # print(kpts)
        kpts = np.array(kpts, copy=False)

        # draw each point on image
        assert len(pose_kpt_color) == len(kpts)

        for kid, kpt in enumerate(kpts):
            x_coord, y_coord = int(kpt[0]), int(kpt[1])

            color = tuple(int(c) for c in pose_kpt_color[kid])
            cv2.circle(img, (int(x_coord), int(y_coord)), radius, color, -1)

        # draw links
        for sk_id, sk in enumerate(skeleton):
            pos1 = (int(kpts[sk[0], 0]), int(kpts[sk[0], 1]))
            pos2 = (int(kpts[sk[1], 0]), int(kpts[sk[1], 1]))

            if (pos1[0] <= 0 or pos1[0] >= img_w or pos1[1] <= 0 or pos1[1] >= img_h or pos2[0] <= 0
                    or pos2[0] >= img_w or pos2[1] <= 0 or pos2[1] >= img_h or pose_link_color[sk_id] is None):
                # skip the link that should not be drawn
                continue
            color = tuple(int(c) for c in pose_link_color[sk_id])
            cv2.line(img, pos1, pos2, color, thickness=thickness)

    return img


import mmcv
from mmdet.apis import inference_detector, init_detector
has_mmdet = True
from mmpose.apis import inference_topdown, init_model as init_pose_estimator

pose_config = '/content/mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w48_8xb32-210e_coco-256x192.py'
pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
det_config = '/content/mmpose/demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

device = 'cuda:0'
cfg_options = dict(model=dict(test_cfg=dict(output_heatmaps=True)))

# build detector
det_model = init_detector(
    det_config,
    det_checkpoint,
    device=device
)


# build pose estimator
pose_model = init_pose_estimator(
    pose_config,
    pose_checkpoint,
    device=device,
    cfg_options=cfg_options
)

#from ldm.util import resize_numpy_image
from mmpose.evaluation.functional import nms
from mmengine.registry import init_default_scope


for path in glob.glob('/content/input_images/*'):
  print(path)
  init_default_scope(det_model.cfg.get('default_scope', 'mmdet'))
  image = cv2.imread(path)

  detect_result = inference_detector(det_model, path)
  pred_instance = detect_result.pred_instances.cpu().numpy()
  bboxes = np.concatenate(
      (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
  bboxes = bboxes[np.logical_and(pred_instance.labels == 0,
                                  pred_instance.scores > 0.3)]
  bboxes = bboxes[nms(bboxes, 0.3)][:, :4]

  # predict keypoints
  pose_results = inference_topdown(pose_model, path, bboxes)

  # show the results
  pose = imshow_keypoints(
      image,
      pose_results,
      radius=2,
      thickness=2)
  
  img = pose
  height, width = img.shape[:2]
  new_width = max(height, width)
  new_height = new_width

  # Create a new image
  new_img = np.zeros((new_height, new_width, 3), dtype=np.uint8)

  # Place the original image in the center of the new image
  if width > height:
      x_offset = 0
      y_offset = int((width - height) / 2)
  else:
      x_offset = int((height - width) / 2)
      y_offset = 0

  new_img[y_offset:y_offset+height, x_offset:x_offset+width] = img


  cv2.imwrite(os.path.join(f'/content/drive/MyDrive/{output_folder_name}/{os.path.basename(path)}'), new_img)